# Rerank with MonoT5

In [1]:
!nvidia-smi

Sun Oct  3 20:49:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:41:00.0 Off |                    0 |
| N/A   19C    P0    49W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
from trectools import TrecRun
import ir_datasets
monoT5Reranker = MonoT5()


DIR='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/'
DIR_v2='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-marco-v2-ecir22/'

2021-10-03 21:01:36 [INFO] loader: Loading faiss with AVX2 support.
2021-10-03 21:01:36 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
2021-10-03 21:01:36 [INFO] loader: Loading faiss.
2021-10-03 21:01:36 [INFO] loader: Successfully loaded faiss.


In [10]:
def load_topics(version, file):
    import pandas as pd
    return pd.read_csv('../../Data/navigational-topics-and-qrels-ms-marco-v' + str(version) + '/' + file, sep='\t', names=['num', 'query'])

df_popular_queries = load_topics(1, 'topics.msmarco-entrypage-popular.tsv')
df_random_queries = load_topics(1, 'topics.msmarco-entrypage-random.tsv')
df_popular_run = TrecRun(DIR + 'entrypage-popular/run.ms-marco-content.bm25-default.txt')
df_random_run = TrecRun(DIR + 'entrypage-random/run.ms-marco-content.bm25-default.txt')

df_popular_queries_v2 = load_topics(2, 'topics.msmarco-v2-entrypage-popular.tsv')
df_random_queries_v2 = load_topics(2, 'topics.msmarco-v2-entrypage-random.tsv')
df_popular_run_v2 = TrecRun(DIR_v2 + 'entrypage-popular/run.msmarco-doc-v2.bm25-default.txt')
df_random_run_v2 = TrecRun(DIR_v2 + 'entrypage-random/run.msmarco-doc-v2.bm25-default.txt')

In [5]:
df_popular_queries

,num,query
0,0,imdb
1,1,canva
2,2,scribd
3,3,best buy
4,4,lenovo
...,...,...
94,94,webmd
95,95,microsoft calendar
96,96,chase finance
97,97,craigslist san francisco bay


In [4]:
df_popular_run

Data from file /mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/entrypage-popular2/run.ms-marco-content.bm25-default.txt

In [6]:
df_random_queries

,num,query
0,0,alumacraft
1,1,Ablebits
2,2,US Beef Corporation
3,3,Legacy Retire
4,4,system id barcode
...,...,...
94,94,Water Works MERRIMACK NH
95,95,Target Copy
96,96,Medicare Enrollment
97,97,inzalo utility systems


In [7]:
df_random_run

Data from file /mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/entrypage-random2/run.ms-marco-content.bm25-default.txt

In [14]:
df_random_run.run_data

,query,q0,docid,rank,score,system
0,0,Q0,D3140592,1,11.010900,Anserini
1,0,Q0,D3140593,2,10.387100,Anserini
2,0,Q0,D3140594,3,9.805400,Anserini
3,0,Q0,D655946,4,9.593400,Anserini
4,0,Q0,D889110,5,9.579000,Anserini
...,...,...,...,...,...,...
86330,98,Q0,D2232058,996,5.429097,Anserini
86331,98,Q0,D3057636,997,5.429096,Anserini
86332,98,Q0,D567026,998,5.428000,Anserini
86333,98,Q0,D2258639,999,5.425000,Anserini


# The actual reranking

In [21]:
def get_query_or_fail(df_queries, topic_number):
    ret = df_queries[df_queries['num'] == int(topic_number)]
    if len(ret) != 1:
        raise ValueError('Could not handle ' + str(topic_number))
    
    return ret.iloc[0]['query']

marco_v1_doc_store = ir_datasets.load('msmarco-document').docs_store()
marco_v2_doc_store = ir_datasets.load('msmarco-document-v2').docs_store()

def get_doc_text(doc_id):
    if doc_id.startswith('msmarco_doc_'):
        ret = marco_v2_doc_store.get(doc_id)
    else:
        ret = marco_v1_doc_store.get(doc_id)
    
    return ret.title + ' ' + ret.body

def docs_for_topic(df_run, topic_number):
    return df_run.run_data[df_run.run_data['query'] == int(topic_number)].docid

def rerank_with_model(topic, df_queries, df_run, model):
    query = get_query_or_fail(df_queries, topic)
    print('rerank query ' + query)
    documents = [Text(get_doc_text(i), {'docid': i}, 0) for i in docs_for_topic(df_run, topic)[:100]]
    ret = sorted(model.rerank(Query(query), documents), key=lambda i: i.score, reverse=True)

    return [{'score': i.score, 'id': i.metadata['docid'], 'body': i.text} for i in ret]

def rerank(file_name, df_run, df_queries, model, tag):
    from tqdm import tqdm
    
    with open(file_name, 'w') as out_file:
        for topic in tqdm(df_queries.num):
            for i in zip(range(100), rerank_with_model(topic, df_queries, df_run, model)):
                out_file.write(str(topic) + ' Q0 ' + i[1]['id'] + ' ' + str(i[0] + 1) + ' ' + str(i[1]['score']) + ' ' + tag + '\n')

# Marco V1

In [54]:
rerank(DIR + 'entrypage-random/run.ms-marco-content.bm25-mono-t5.txt', df_random_run, df_random_queries.copy(), monoT5Reranker, 'mono-t5-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query alumacraft


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:08<13:27,  8.24s/it]

rerank query Ablebits


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:10<10:27,  6.47s/it]

...

/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 98%|█████████▊| 97/99 [14:52<00:18,  9.23s/it]

rerank query inzalo utility systems


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 99%|█████████▉| 98/99 [15:00<00:08,  8.79s/it]

rerank query Diana Jules Real Estate


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 99/99 [15:04<00:00,  9.14s/it]


In [59]:
rerank(DIR +  'entrypage-popular/run.ms-marco-content.bm25-mono-t5.txt', df_popular_run, df_popular_queries.copy(), monoT5Reranker, 'mono-t5-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query imdb


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:04<07:36,  4.66s/it]

rerank query canva


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:07<06:49,  4.23s/it]

...

/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 98%|█████████▊| 97/99 [10:45<00:09,  4.83s/it]

rerank query craigslist san francisco bay


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 99%|█████████▉| 98/99 [10:55<00:06,  6.26s/it]

rerank query Microsoft Power BI


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 99/99 [11:03<00:00,  6.70s/it]


# Marco V2

In [16]:
rerank(DIR_v2 + 'entrypage-random/run.ms-marco-content.bm25-mono-t5.txt', df_random_run_v2, df_random_queries_v2.copy(), monoT5Reranker, 'mono-t5-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query alumacraft


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:09<15:43,  9.63s/it]

rerank query Ablebits


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:12<09:13,  5.71s/it]

...

 98%|█████████▊| 97/99 [14:27<00:19,  9.64s/it]

rerank query inzalo utility systems


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 99%|█████████▉| 98/99 [14:40<00:10, 10.66s/it]

rerank query Diana Jules Real Estate


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 99/99 [14:50<00:00,  8.99s/it]


In [17]:
rerank(DIR_v2 +  'entrypage-popular/run.ms-marco-content.bm25-mono-t5.txt', df_popular_run_v2, df_popular_queries_v2.copy(), monoT5Reranker, 'mono-t5-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query imdb


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:07<12:39,  7.75s/it]

rerank query canva


  2%|▏         | 2/99 [00:16<13:13,  8.18s/it]

...

/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 98%|█████████▊| 97/99 [13:25<00:11,  5.51s/it]

rerank query craigslist san francisco bay


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 99%|█████████▉| 98/99 [13:32<00:05,  5.88s/it]

rerank query Microsoft Power BI


100%|██████████| 99/99 [13:36<00:00,  8.25s/it]


# Rerank with MonoBERT

In [19]:
from pygaggle.rerank.transformer import MonoBERT

monoBert = MonoBERT()

In [ ]:
rerank(DIR + 'entrypage-random/run.ms-marco-content.bm25-mono-bert.txt', df_random_run, df_random_queries.copy(), monoBert, 'mono-bert-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query alumacraft


  1%|          | 1/99 [00:06<10:39,  6.53s/it]

rerank query Ablebits


  2%|▏         | 2/99 [00:08<06:12,  3.84s/it]

...

rerank query Jamestown Tennessee


 55%|█████▍    | 54/99 [09:42<08:52, 11.83s/it]

rerank query PANDAS Network


 56%|█████▌    | 55/99 [09:51<07:51, 10.73s/it]

rerank query UN


In [23]:
rerank(DIR +  'entrypage-popular/run.ms-marco-content.bm25-mono-bert.txt', df_popular_run, df_popular_queries.copy(), monoBert, 'mono-bert-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]

rerank query imdb


  1%|          | 1/99 [00:11<18:47, 11.51s/it]

rerank query canva


  2%|▏         | 2/99 [00:21<17:00, 10.52s/it]

...

 98%|█████████▊| 97/99 [18:12<00:16,  8.39s/it]

rerank query craigslist san francisco bay


 99%|█████████▉| 98/99 [18:28<00:10, 10.66s/it]

rerank query Microsoft Power BI


100%|██████████| 99/99 [18:40<00:00, 11.32s/it]


In [ ]:
rerank(DIR_v2 + 'entrypage-random/run.ms-marco-content.bm25-mono-bert.txt', df_random_run_v2, df_random_queries_v2.copy(), monoBert, 'mono-bert-at-bm25')

In [ ]:
rerank(DIR_v2 +  'entrypage-popular/run.ms-marco-content.bm25-mono-bert.txt', df_popular_run_v2, df_popular_queries_v2.copy(), monoBert, 'mono-bert-at-bm25')